In [1]:
from pathlib import Path

import fitz

import pandas as pd 
import re


In [13]:
def extrair_elementos_docling(folder_path):
    """Extrai textos de todos PDFs dentro de uma pasta (não recursivo)"""
    folder = Path(folder_path)
    texts = []
    pdf_files = list(folder.glob("*.pdf"))
    for pdf_file in pdf_files:
        print(f"Lendo arquivo: {pdf_file.name}")
        doc = fitz.open(str(pdf_file))
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        texts.append((pdf_file, text))
    return texts

def parse_notas(textos):
    dados = []
    for filename, texto in textos:
        # Extrair cabeçalho
        turma = re.search(r"Turma\s+(\w+)", texto)
        disciplina = re.search(r"^Disciplina\s*\n(.+)$", texto, re.MULTILINE)
        ano_letivo = re.search(r"Ano Letivo\s+(\d{4})", texto)

        turma = turma.group(1) if turma else None
        disciplina = disciplina.group(1).strip() if disciplina else None
        ano_letivo = int(ano_letivo.group(1)) if ano_letivo else None

        # Regex para alunos
        padrao = re.compile(
            r"\b(\d+)\s+"                           # ordem
            r"(\d{10}-\d)\s+"                       # matrícula
            r"([A-ZÀ-ÿ\s]+?)\s+"                    # nome
            r"(-|[\d,]+)\s+"                        # nota 1
            r"(-|[\d,]+)\s+"                        # nota 2
            r"(-|[\d,]+)\s+"                        # nota 3
            r"(-|[\d,]+)\s+"                        # nota 4
            r"(-|[\d,]+)\s+"                        # para fechar
            r"(Aprovado|Reprovado|\(transferido\))",  # status
            flags=re.MULTILINE
        )

        for match in re.findall(padrao, texto):
            _, matricula, nome, n1, n2, n3, n4, fechar, status = match
            dados.append({
                #"arquivo": filename,
                "turma": turma,
                "ano_letivo": ano_letivo,
                "matricula": matricula,
                "aluno": nome.strip().title(),
                "disciplina": disciplina,
                "nota_1bim": float(n1.replace(',', '.')) if n1 != '-' else None,
                "nota_2bim": float(n2.replace(',', '.')) if n2 != '-' else None,
                "nota_3bim": float(n3.replace(',', '.')) if n3 != '-' else None,
                "nota_4bim": float(n4.replace(',', '.')) if n4 != '-' else None,
                "para_fechar": float(fechar.replace(',', '.')) if fechar != '-' else None,
                "status": status
            })

    return pd.DataFrame(dados)


In [14]:
path = Path().resolve()
current_path = path / "bases/notas"
current_path

WindowsPath('C:/Users/user/Documents/projetos/python/tcc/projeto/bases/notas')

In [15]:
texto = extrair_elementos_docling(current_path)
df = parse_notas(texto)
df.to_csv("Notas_alunos.csv", index=False)

Lendo arquivo: 1A_ Mapa de notas_ Arte.pdf
Lendo arquivo: 1A_ Mapa de notas_ Biologia.pdf
Lendo arquivo: 1A_ Mapa de notas_ Educação Física.pdf
Lendo arquivo: 1A_ Mapa de notas_ Filosofia.pdf
Lendo arquivo: 1A_ Mapa de notas_ Física.pdf
Lendo arquivo: 1A_ Mapa de notas_ Geografia.pdf
Lendo arquivo: 1A_ Mapa de notas_ História.pdf
Lendo arquivo: 1A_ Mapa de notas_ Inglês.pdf
Lendo arquivo: 1A_ Mapa de notas_ Matemática.pdf
Lendo arquivo: 1A_ Mapa de notas_ Português.pdf
Lendo arquivo: 1A_ Mapa de notas_ Química.pdf
Lendo arquivo: 1A_ Mapa de notas_ Sociologia.pdf


In [16]:
df

,turma,ano_letivo,matricula,aluno,disciplina,nota_1bim,nota_2bim,nota_3bim,nota_4bim,para_fechar,status
0,1A,2024,2412752317-6,Ana Clara Reis Da Silva,ARTE,7.4,8.4,8.2,8.0,0.0,Aprovado
1,1A,2024,2412727121-1,Ana Julia Da Silva E Silva,ARTE,6.2,8.6,6.2,7.8,0.0,Aprovado
2,1A,2024,1811397482-9,Anna Clara Calero Guida,ARTE,7.3,6.9,6.8,6.3,0.0,Aprovado
3,1A,2024,2412732617-5,Dérick Kauã Lima Ferreira,ARTE,7.0,8.2,7.6,8.3,0.0,Aprovado
4,1A,2024,2312711135-2,Eliaeste Gomes Sardinha Neto,ARTE,6.4,7.8,6.4,7.8,0.0,Aprovado
...,...,...,...,...,...,...,...,...,...,...,...
331,1A,2024,2112128423-8,Rhuan Carlos Galeno Da Silva,SOCIOLOGIA,6.3,6.9,6.2,6.1,0.0,Aprovado
332,1A,2024,2412760783-3,Sheila Cindy Guarachi Condori,SOCIOLOGIA,7.3,6.7,7.6,6.1,0.0,Aprovado
333,1A,2024,2212247310-0,Thyeska Cristini Machado,SOCIOLOGIA,NaN,NaN,NaN,NaN,NaN,(transferido)
334,1A,2024,2011788460-8,Victor Daniel Oliveira Pereira,SOCIOLOGIA,6.2,6.2,6.2,6.1,0.0,Aprovado
